In [10]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
import random

In [11]:
import geo_utils

In [13]:
import urllib2

__HOST__ = r'http://lcmap-test.cr.usgs.gov/changes/results'
__ALGORITHM__ = r'lcmap-pyccd:1.1.0'


def api_request(x, y, refresh=False):
    endpoint = '/'.join([__HOST__, __ALGORITHM__, str(x), str(y)]) + '?refresh={}'.format(str(refresh).lower())
    
    request = urllib2.Request(endpoint)

    #     base64string = (base64
    #                     .encodestring('%s:%s' % (username, password))
    #                     .replace('\n', ''))
    #     request.add_header("Authorization", "Basic %s" % base64string)

    try:
        result = urllib2.urlopen(request)
    except urllib2.HTTPError as e:
        result = e

    return json.loads(result.read())

In [16]:
def request_tile(h, v, refresh=False):
    ext, _ = geo_utils.extent_from_hv(h, v)
    
    resps = []
    for y in xrange(ext.y_max, ext.y_max - 5000 * 30, -3000):
        for x in xrange(ext.x_min, ext.x_min + 5000 * 30, 3000):
            resps.append(api_request(x, y, refresh=refresh))

In [15]:
request_tile(2, 9, True)

(1964805, -2265585)
